In [1]:
import tensorflow as tf

변수(Variable)와 플레이스홀더(placeholder) 를 이용하여
선형회귀 모델의 그래프 구조 (Wx + b) 선언
- W: 가중치
- b: 바이어스
- x: 입력값

In [2]:
W = tf.Variable(tf.random_normal(shape=[1]))
b = tf.Variable(tf.random_normal(shape=[1]))
x = tf.placeholder(tf.float32)
linear_model = W*x + b

Instructions for updating:
Colocations handled automatically by placer.


- y: Output

In [3]:
y = tf.placeholder(tf.float32)

Loss Function: MSE 선언

In [4]:
loss = tf.reduce_mean(tf.square(linear_model-y))

Optimier 선언: GradientDescent

In [5]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train_step = optimizer.minimize(loss)

Instructions for updating:
Use tf.cast instead.


실제 Data Input

In [6]:
X_train = [1, 2, 3, 4]
Y_train = [2, 4, 6, 8]

Session 을 통하여 변수 초기화 및 Graph 그리기

In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(1000):
    sess.run(train_step,feed_dict={x:X_train, y:Y_train})

Model Test 및 Session 닫기

In [8]:
X_test = [3.5, 5, 5.5, 6]
print(sess.run(linear_model,feed_dict={x:X_test}))

sess.close()

[ 7.0019174 10.007053  11.008765  12.010476 ]


### TensorBoard
텐서플로 라이브러리는 학습 결과를 시각화해서 보여주는 강력한 도구인 텐서보드(Tensorboard)를 제공한다.
이를 통하여 학습이 올바른 방향으로 진행돠고 있는지를 측정하거나 실험결과를 분석하는 데 많은 도움이 된다.
아래 예시는 Tensorboard에서 가장많이 활용되는 예시 2개이다.
**Tensorboard의 활용예시**
1. ts.summary.scalar API를 이용한 step마다 손실함수 출력
2. 계산 그래프 시각화

#### 1. tf.summary.scalar API 활용 예시
변수선언

In [9]:
W = tf.Variable(tf.random_normal(shape=[1]), name = 'W')
b = tf.Variable(tf.random_normal(shape=[1]), name = 'b')
x = tf.placeholder(tf.float32, name = 'x')
linear_model = W*x + b
y = tf.placeholder(tf.float32, name = 'y')

- loss 선언
- **텐서보드를 위한 요약정보(scalar)선언 => 요약 정보 로그를 남기기 위해**

In [10]:
loss = tf.reduce_mean(tf.square(linear_model-y))
#텐서 보드를 위한 요약정보 scalar
tf.summary.scalar('loss',loss)

<tf.Tensor 'loss:0' shape=() dtype=string>

- tf.summary.merge_all: 모든 요약정보를 하나로 합치는 연산
- tf.summary.FileWriter: 요약정보를 FIle형태로 저장하는 것

In [11]:
merged = tf.summary.merge_all()
tensorboard_writer = tf.summary.FileWriter('./tensorboard_log2',sess.graph)

Session 실행 및 Step마다 merged연산을 실행하고 저장

In [12]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train_step = optimizer.minimize(loss)

X_train = [1, 2, 3, 4]
Y_train = [2, 4, 6, 8]

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(1000):
    sess.run(train_step,feed_dict={x:X_train, y:Y_train})
    
    #매 Step마다 요약정보 값들을 저장하여 지정된 경로에 저장
    summary = sess.run(merged,feed_dict={x:X_train, y:Y_train})
    tensorboard_writer.add_summary(summary,i)
    
sess.close()